# Builder Tutorial number 8

The builder tutorials demonstrate how to build an operational GSFLOW model using `pyGSFLOW` from shapefile, DEM, and other common data sources. These tutorials focus on the `gsflow.builder` classes.

## Building the GSFLOW control file

In this tutorial, we demostrate how to build the GSFLOW control file. The building method relies on a number of defaults stored within pyGSFLOW, which allows the user to 1) rapidly generate a basic PRMS model, and 2) later adjust it using pyGSFLOW built in functionality. 

In [1]:
import os
import utm
import shapefile
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import flopy
from gsflow.builder import GenerateFishnet, FlowAccumulation
import gsflow

### The `ControlFileBuilder` class

The `ControlFileBuilder` class builds a GSFLOW control file from pyGSFLOW default parameter values.

The `ControlFileBuilder` class has a single optional parameter:
   - `defaults` : optional parameter, user can supply a gsflow.builder.Defaults instance to ModflowBuilder to use a custom set of default values
   
Let's start by importing the class

In [2]:
from gsflow.builder import ControlFileBuilder

## Applying the methods to the Sagehen 50m example problem

In this example the methods are applied directly to the Sagehen 50m model as they are presented.

**Let's start by loading the Modflow model and the PRMS parameter file that was produced in the previous two builder tutorials**

In [3]:
# define the input and output data paths
input_ws = os.path.join("data", "sagehen", "50m_tutorials")
output_ws = os.path.join("data", "temp")

modflow_nam = "sagehen_50m.nam"
parameter_file = os.path.join(input_ws, "sagehen_50m_initial.param")

Load the modflow model and parameter file

In [4]:
ml = gsflow.modflow.Modflow.load(modflow_nam, model_ws=input_ws)
parameters = gsflow.prms.PrmsParameters.load_from_file(parameter_file)

   loading iuzfbnd array...
   loading irunbnd array...
   loading vks array...
   loading eps array...
   loading thts array...
stress period 1:
   loading finf array...
stress period 2:
------------------------------------
Reading parameter file : sagehen_50m_initial.param
------------------------------------


### Building a control file object

Now that the modflow model and the prms parameter object have been loaded, `ControlFileBuilder` can be instantiated and a pyGSFLOW `ControlFile` object can be created containing the path information for the modflow model and the PRMS parameter file(s).

For the Sagehen 50m model built in `Defaults` will be used create the initial iteration of the control file.

In [5]:
controlbuild = ControlFileBuilder()

To build a control file object the user need to call the `build()` method. The `build()` method has three optional parameters:
   - `name` : model name, defaults to "gsflow_builder"
   - `parameter_obj` : if a `PrmsParameters` object is supplied, the control file builder will set the name(s) of the parameter file(s) in the control file
   - `modflow_obj` : if a `Modflow` object is supplied, the control file builder will set the name of the 'modflow_name' parameter in the control file.
   
In this example we use all three optional parameters

In [6]:
control = controlbuild.build("sagehen_50m", parameters, ml)
print(type(control))

<class 'gsflow.control.ControlFile'>


## `ControlFile` object overview

The `ControlFile` object holds parameter information and allows the user to edit parameters, add parameters, remove parameters, and write control file parameter information. The `pygsflow_quickstart` notebooks show how to interact with this class in more detail than will be presented here.

Instead this example shows how to add a record and how to update the values of an existing record in the `ControlFile` object.

Futher setup is presented in a later notebook.

### `ControlRecord` explanation

All control file parameter values are stored in a `ControlRecord` object that contains information about the parameter's data type and number of values. Let's look at a `ControlRecord` object quickly

In [7]:
mf_name = control.get_record("modflow_name")

print(mf_name)
print(type(mf_name))
print(mf_name.values)


####
modflow_name
1
4
sagehen_50m.nam
####
<class 'gsflow.control.ControlRecord'>
['sagehen_50m.nam']


### Updating the values in an existing control file parameter

The `ControlFile` object allows the user to easily update existing parameters using pyGSFLOW. Here is an example of updating the "model_mode" and the "srunoff_module" parameters.

Because of python overloading methods setting values can be applied to the `ControlRecord` object by calling it's name as an attribute.

In [8]:
control.model_mode = ["GSFLOW5",]

print(control.model_mode)


####
model_mode
1
4
GSFLOW5
####


In [9]:
control.srunoff_module = ["srunoff_smidx",]

print(control.srunoff_module)


####
srunoff_module
1
4
srunoff_smidx
####


### Writing the `ControlFile` object to a GSFLOW control file 

This section shows how to write the `ControlFile` object to a GSFLOW control file using the built in `write()` method. The Builder Tutorial series will revist this file later for further tuning before creating the finished Sagehen 50m model.

In [10]:
control_file = os.path.join(output_ws, "sagehen_50m_initial.control")
control.write(control_file)